---
# Tutoriel 2 - Méthodes pratiques
---

<center><img src="./img/mlprocess_3.png" alt="Processus d'apprentissage automatique" width="50%"/></center>

Dans ce tutoriel, nous ferons une introduction à la librairie Poutyne qui facilite l'entraînement de réseaux de neurones en PyTorch. 

1. Introduction à Poutyne
    1. Boucle d'entraînement, notion de ```Callbacks``` (```Logging```, ```WeightViz```, etc.)
2. Visualisation des poids avec le callback ```WeightViz```
    1. Introduction à TensorBoard
3. Programmation d'un réseau multicouche
    1. Fonctions d'activation
4. Initialisation des poids
5. Horaire d'entraînement (```Learning Rate```)
6. Régularisation
7. Ajout du callback ```EarlyStopping```
9. Résultats sur l'ensemble de test


## 1. Introduction à Poutyne

In [1]:
import math
import torch
import numpy as np
from torch import optim, nn
from torchvision.datasets.mnist import MNIST
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from poutyne.framework import Model, ModelCheckpoint, Callback, CSVLogger, EarlyStopping, ReduceLROnPlateau
from poutyne import torch_to_numpy

torch.manual_seed(42)
np.random.seed(42)

ModuleNotFoundError: No module named 'torch.utils.tensorboard'

In [3]:
def load_mnist(download=False, path='./', transform=None):
    """Loads the MNIST dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train and test dataset
    """
    train_dataset = MNIST(path, train=True, download=download, transform=transform)
    test_dataset = MNIST(path, train=False, download=download, transform=transform)
    return train_dataset, test_dataset


def load_mnist_with_validation_set(download=False, path='./', train_split=0.8):
    """Loads the MNIST dataset.

    :param download: Download the dataset
    :param path: Folder to put the dataset
    :return: The train, valid and test dataset ready to be ingest in a neural network
    """
    train, test = load_mnist(download, path, transform=ToTensor())
    lengths = [round(train_split*len(train)), round((1.0-train_split)*len(train))]
    train, valid = random_split(train, lengths)
    return train, valid, test

def count_number_of_parameters(net):
    return sum(p.numel() for p in net.parameters() if p.requires_grad)

In [ ]:
# Hyperparamètres d'entraînement
cuda_device = 0
device = torch.device("cuda:%d" % cuda_device if torch.cuda.is_available() else "cpu")
batch_size = 32
learning_rate = 0.01
n_epoch = 5
num_classes = 10

In [ ]:
train, valid, test = load_mnist_with_validation_set(download=True)

In [ ]:
# Tailles des ensembles d'entraînement, de validation et de test.
len(train), len(valid), len(test)

Nous définissons des chargeurs de données (`dataloaders`) qui vont nous fournir `batch_size` images à la fois, c'est-à-dire des lots (`batchs`) de taille `batch_size`. Nous avons 3  chargeurs de données pour les 3 ensembles utilisés: entraînement, validation et test.

In [ ]:
train_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid, batch_size=batch_size)
test_loader = DataLoader(test, batch_size=batch_size)

Le code suivant initialise un réseau identique à celui du Tutoriel 1 (voir tableau ci-dessous), mais en utilisant la méthode séquentielle de PyTorch. Cette méthode permet de voir notre réseau en fonction de ses différentes couches. Il est à noter qu'il n'est pas toujours possible d'utiliser cette méthode.

| Type de couche              | Taille de sortie |      # de paramètres   |
|-----------------------------|:----------------:|:----------------------:|
| Input                       |   1x28x28   |              0            |
| Flatten                     |  1\*28\*28  |              0            |
| **Linear avec 10 neurones** |     10      | 28\*28\*10 + 10 = 7 850 |

\# total de paramètres du réseau: 7 850

In [ ]:
def create_simple_fully_connected_network():
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, num_classes)
    )

La fonction ci-dessous effectue l'entraînement d'un réseau de neurones avec Poutyne. Pour ce faire, nous utilisons la classe Model de Poutyne qui est analogue à la classe Trainer que nous avions définie dans le tutoriel 1. Par contre, comme nous allons le voir, la classe Model offre beaucoup plus de fonctionnalités.


In [ ]:
def train(name, network):
    print(network)
    optimizer = optim.SGD(network.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()

    # Objet Model de Poutyne
    model = Model(network, optimizer, loss_function, batch_metrics=['accuracy'])

    # Envoi du modèle sur GPUs si disponibles
    model.to(device)

    # Lancement de l'entraînement
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch)

In [ ]:
net = create_simple_fully_connected_network()
train('fc_simple', net)

## 2. Visualisation des poids

Une des fonctionnalités essentielles de Poutyne sont les [callbacks](https://poutyne.org/callbacks.html). Un callback permet d'effectuer des actions pendant l'entraînement. L'exemple ci-dessous permet d'écrire dans [TensorBoard](https://www.tensorflow.org/tensorboard/) les poids de notre réseau pour que nous puissions regarder l'évolution de ceux-ci pendant l'entraînement.

In [ ]:
class WeightsVisualizer(Callback):
    def __init__(self, tensorboard_writer):
        """
        Callback écrivant les poids dans TensorBoard à chaque début d'époque.
        
        Args:
            tensorboard_writer (SummaryWriter): Objet pour écrire dans TensorBoard.
        """
        super(WeightsVisualizer, self).__init__()
        self.writer = tensorboard_writer

    def on_epoch_begin(self, epoch, logs):
        # À chaque début d'époque, nous allons chercher les poids de la couche 1.
        weights = self.model.network[1].weight.view(-1, 1, 28, 28)
        # Et nous les écrivons dans TensorBoard.
        self.writer.add_image('weights', make_grid(weights, nrow=5), global_step=epoch)

In [ ]:
def train_visualizer(name, network):
    print(network)
    optimizer = optim.SGD(network.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()
    
    # Définition de l'objet permettant d'écrire dans TensorBoard.
    writer = SummaryWriter('runs/')
    
    # Définition de la liste des callbacks. 
    # C'est une liste parce qu'il est possible d'en passer plusieurs.
    # Nous instancions le callback défini précédemment.
    callbacks = [WeightsVisualizer(writer)]

    # Objet Model de Poutyne
    model = Model(network, optimizer, loss_function, batch_metrics=['accuracy'])

    # Envoi du modèle sur GPUs si disponibles
    model.to(device)

    # Lancement de l'entraînement
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch, callbacks=callbacks)

In [ ]:
learning_rate = 0.0005
net = create_simple_fully_connected_network()
train_visualizer('fc_simple_visualizer', net)

## 3. Programmation d'un réseau multicouche

Nous allons augmenter le nombre de couches du réseau. En augmentant le nombre de couches, on augmente la capacité de notre réseau, à condition d'utiliser des fonctions d'activation! Implémentez le réseau suivant en utilisant la manière séquentielle. Un début de code vous est fourni.

| Type de couche              | Taille de sortie |      # de paramètres   |
|-----------------------------|:----------------:|:----------------------:|
| Input                       |   1x28x28   |              0              |
| Flatten                     |  1\*28\*28  |              0              |
| **Linear with 256 neurons** |     256     | 28\*28\*256 + 256 = 200 960 |
| ReLU                        |      *      |              0              |
| **Linear with 128 neurons** |     128     |   256*128 + 128 = 32 896    |
| ReLU                        |      *      |              0              |
| **Linear with 64 neurons**  |     64      |     128*64 + 64 = 8 256     |
| ReLU                        |      *      |              0              |
| **Linear with 10 neurons**  |     10      |      64*10 + 10 = 650       |

\# total de paramètres du réseau: 242 762


In [ ]:
def create_fully_connected_network():
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(28*28, 256),
        nn.ReLU(),
        ... # À faire: complétez le réseau selon le tableau ci-dessus.
    )

In [ ]:
def train_mlp(name, network):
    print(network)
    optimizer = optim.SGD(network.parameters(), lr=learning_rate)
    loss_function = nn.CrossEntropyLoss()

    # Objet Model de Poutyne
    model = Model(network, optimizer, loss_function, batch_metrics=['accuracy'])

    # Envoi du modèle sur GPUs si disponibles
    model.to(device)

    # Lancement de l'entraînement
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch)

In [ ]:
learning_rate = 0.01
net = create_fully_connected_network()
train_mlp('mlp', net)

In [ ]:
count_number_of_parameters(net)

## 4. Initialisation des poids des neurones

L'initialisation des poids permet une convergence plus rapide dans la majorité des cas.

In [ ]:
net = create_fully_connected_network()
named_parameters = list(net.named_parameters())

In [ ]:
named_parameters[0]

In [ ]:
nn.init.kaiming_normal_(named_parameters[0][1])

In [ ]:
def init_weights(net):
    for name, params in net.named_parameters():
        if 'weight' in name:
            nn.init.kaiming_normal_(params)
        elif 'bias' in name:
            nn.init.constant_(params, 0)
init_weights(net)

In [ ]:
named_parameters[0], named_parameters[1]

In [ ]:
train_mlp('mlp', net)

## 5. Régularisation de l'entraînement

La régularisation est utilisée pour prévenir le surapprentissage (overfitting). Dans l'exemple ci-dessous, nous utilisons l'argument `weight_decay` de l'optimiseur pour activer la régularisation de type L2.


Analysons d'abord la norme des poids.  C'est un indicateur de l'espace possible d'un réseau de neurones.


In [ ]:
torch.norm(named_parameters[0][1], 2)

In [ ]:
def train_mlp_reg(name, network):
    print(network)
    optimizer = optim.SGD(network.parameters(), lr=learning_rate, weight_decay=0.01)
    loss_function = nn.CrossEntropyLoss()

    # Objet Model de Poutyne
    model = Model(network, optimizer, loss_function, batch_metrics=['accuracy'])

    # Envoi du modèle sur GPUs si disponibles
    model.to(device)

    # Lancement de l'entraînement
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch)

In [ ]:
net = create_fully_connected_network()
init_weights(net)
train_mlp_reg('mlp_reg', net)

In [ ]:
named_parameters = list(net.named_parameters())
torch.norm(named_parameters[0][1], 2)

## 6. Early Stopping et horaire d'entraînement

Comme vu lors des présentations, le `early stopping` ainsi que les horaires d'entraînement peuvent nous aider à éviter le surapprentissage. Poutyne offre un callback de type [EarlyStopping](https://poutyne.org/callbacks.html#poutyne.framework.callbacks.EarlyStopping) et un callback de type [ReduceLROnPlateau](https://poutyne.org/callbacks.html#poutyne.framework.callbacks.lr_scheduler.ReduceLROnPlateau) pour faciliter l'utilisation de ces méthodes. Il est à noter que le callback `ReduceLROnPlateau` encapsule tout simplement une classe de PyTorch afin d'en faciliter l'utilisation.


Complétez le code ci-dessous en instanciant un callback de type [EarlyStopping](https://poutyne.org/callbacks.html#poutyne.framework.callbacks.EarlyStopping) avec une patience de *5* et un callback de type [ReduceLROnPlateau](https://poutyne.org/callbacks.html#poutyne.framework.callbacks.lr_scheduler.ReduceLROnPlateau) avec une patience de *2*. Ajoutez l'argument `verbose=True` pour afficher les callbacks exécutés lors de l'entraînement.

In [ ]:
def train_mlp_early_stopping(name, network):
    optimizer = optim.SGD(network.parameters(), lr=learning_rate, weight_decay=0.001)
    loss_function = nn.CrossEntropyLoss()
    
    early_stopping = ... # À faire: instancier un callback de type EarlyStopping
    lr_scheduler = ... # À faire: instancier un callback de type ReduceLROnPlateau
    callbacks = [early_stopping, lr_scheduler]

    # Objet Model de Poutyne
    model = Model(network, optimizer, loss_function, batch_metrics=['accuracy'])

    # Envoi du modèle sur GPUs si disponibles
    model.to(device)

    # Lancement de l'entraînement
    model.fit_generator(train_loader, valid_loader, epochs=n_epoch, callbacks=callbacks)

In [ ]:
n_epoch = 100
net = create_fully_connected_network()
init_weights(net)
train_mlp_early_stopping('mlp_early_stopping', net)

## 7. Évaluation du modèle final

Créez votre propre modèle pour qu'il soit le meilleur possible sur l'ensemble de validation. Lorsque vous êtes fin prêts, évaluer la performance de celui-ci sur l'ensemble de test!

In [ ]:
# Créez votre réseau
net = ...

# Appliquez quelques méthodes que nous avons vues
optimizer = ...
n_epoch = 10
loss_function = nn.CrossEntropyLoss()
callbacks = []

model = Model(net, optimizer, loss_function, batch_metrics=['accuracy'])
model.to(device)

# Train
model.fit_generator(train_loader, valid_loader, epochs=n_epoch, callbacks=callbacks)

Seulement lorsque vous êtes prêt!

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_loader)
print('Test:\n\tLoss: {}\n\tAccuracy: {}'.format(test_loss, test_acc))